In [13]:
from astropy.table import join
from astropy.table import Table
from astropy.table import  hstack
from astropy.table import Column
from astropy import units as u
from astropy.coordinates import SkyCoord


from CRTS_paper_modules import update_progress
import CRTS_paper_modules as mod
import numpy as np
import matplotlib.pyplot as plt 
from os import listdir
import pandas as pd 
import imp
imp.reload(mod)


<module 'CRTS_paper_modules' from '/Users/chris/GradResearch/CRTS_PROJECT/code/CRTS_paper_modules.py'>

# CRTS stars 

Aim : want to figure out WHY  all SDSS and CRTS stars can be matched to matching radius of 0.0 ! How is this possible? Are CRTS RA even different ? 
    

In [2]:
# CRTS stars radec

address = '../raw_LC_CRTS/stars/radec.00'
crts_stars_radec = Table.read(address, format='ascii', names=['CRTS_name','ra','dec'])


In [3]:
len(crts_stars_radec)


100000

In [11]:
# SDSS stars

address = '../catalogs_SDSS/stripe82calibStars_v2.6.dat' 
colnames = ['calib_fla', 'ra', 'dec', 'raRMS', 'decRMS', 'nEpochs', 'AR_val', 
                'u_Nobs', 'u_mMed', 'u_mMean', 'u_mErr', 'u_rms_scatt', 'u_chi2',
                'g_Nobs', 'g_mMed', 'g_mMean', 'g_mErr', 'g_rms_scatt', 'g_chi2',
                'r_Nobs', 'r_mMed', 'r_mMean', 'r_mErr', 'r_rms_scatt', 'r_chi2',
                'i_Nobs', 'i_mMed', 'i_mMean', 'i_mErr', 'i_rms_scatt', 'i_chi2',
                'z_Nobs', 'z_mMed', 'z_mMean', 'z_mErr', 'z_rms_scatt', 'z_chi2']
    
sdss_stars = Table.read(address, format='ascii', names = colnames)
    

In [17]:
np.in1d(crts_stars_radec['ra'],sdss_stars['ra'])

array([ True,  True,  True, ...,  True,  True,  True], dtype=bool)

In [15]:
len(crts_stars_radec)

100000

 We combine aggregate info of raw LC and proc LC together with SDSS info, using the ra, dec which are for SDSS

In [18]:
# read in CRTS aggregate info  for stars 
address = '../data_products/CRTS_stars_raw_and_proc_combined_agg.dat'
crts_stars = Table.read(address, format='ascii')

In [26]:
# Add the SDSS ra, dec information.... 
crts_table = join(left=crts_stars, right = crts_stars_radec, keys='CRTS_name')

# It's length is the same as the crts_stars :  48250 / 100000  stars, that fulfill the criteria 
# of being longer than 10 day-averaged epochs 

In [33]:
# Check that indeed all ra are in the SDSS (because they are the same )

np.sum(np.in1d(crts_table['ra'], sdss_stars['ra']))

# Indeed they are! 

48250

In [52]:
len(np.unique(crts_table['dec']))

47748

In [ ]:
# Just join the SDSS info to CRTS table 
#crts_sdss_table = join(left=crts_table, right=sdss_stars )

# Interestingly , for 48250 stars, there are only 47137 unique ra, and 47748 unique dec, i.e. some stars have either 
# the same ra, or the same dec. 

# So it seems that there are 820150 unique dec  , and 656841  unique ra  in sdss_stars.
# even if we join on both 'ra' and 'dec', it seems that still we are getting more than one match from SDSS for
# few stars . Just try using astropy catalog match, and see if it works perhaps better.... 

In [53]:
# Astropy  SkyCoord.match_to_catalog_sky 

cat1_ra  = crts_table['ra']
cat1_dec = crts_table['dec']
cat2_ra  = sdss_stars['ra']
cat2_dec = sdss_stars['dec']

from astropy import units as u
from astropy.coordinates import SkyCoord
cat1 = SkyCoord(ra=cat1_ra*u.degree, dec=cat1_dec*u.degree)
cat2 = SkyCoord(ra=cat2_ra*u.degree, dec=cat2_dec*u.degree)
idx, sep2d, dist3d = cat1.match_to_catalog_sky(cat2) 
#idx : indices that match catalog2  to catalog1 : 
#        catalog1[ra]  - catalog2[ra][idx] ~= 0 
# eg. crts_table['dec'] - sdss_stars[idx]['dec'] 

SyntaxError: 'return' outside function (<ipython-input-53-436616adb299>, line 13)

In [76]:
# Save the combined CRTS - SDSS table for stars... 

address = '../data_products/CRTS_SDSS_combined_stars_catalog.dat'
sdss_crts_table.write(address, format='ascii', overwrite='True')

# CRTS quasars  matching to SDSS

Here I use CRTS aggregates on raw and processed lightcurves, and obtain ra,dec information from the filename.  I convert the hms to deg in two  ways:  using a handwritten routine, and using AstroPy SkyCoord.  Both yield exactly the same results : 15 QSO without a counterpart within 50 arcsec... 

In [14]:
# Use the aggregate info on raw and proc CRTS lightcurves, obtain the CRTS ra, dec from filename,
# Read in the raw and processed CRTS LC aggregates 

address = '../data_products/CRTS_qso_raw_and_proc_combined_agg.dat'
crts_qso = Table.read(address, format='ascii')

In [15]:


# extract ra, dec from  CRTS_name   : 
ra_string = [name[:-9] for name in  crts_qso['CRTS_name']]
dec_string  = [name[9:] for name in crts_qso['CRTS_name']]


# Split CRTS  ra, dec from hms to h m s 
ra_hms_split, dec_hms_split = mod.get_ra_dec_CRTS(ra_string, dec_string)

# Convert CRTS  ra, dec from hms to deg  
ra_deg_CRTS, dec_deg_CRTS = mod.convert_to_deg(ra_hms_split, dec_hms_split)


# add that as two columns :
ra = Column(name='ra_CRTS', data=ra_deg_CRTS)
crts_qso.add_column(ra)

dec = Column(name='dec_CRTS', data=dec_deg_CRTS)
crts_qso.add_column(dec)

# Load SDSS Quasars 

File = '../catalogs_SDSS/DB_QSO_S82.dat'
colnames = ['dbID', 'ra', 'dec', 'SDR5ID', 'M_i', 'M_i_corr', 'redshift',
                   'mass_BH', 'Lbol', 'u', 'g', 'r', 'i', 'z', 'Au']
sdss_qso = Table.read(File, format='ascii', names = colnames)

use_astropy_angles = False 
if use_astropy_angles == True : 
    CRTS_coords = np.zeros_like(ra_hms_split, dtype='<U31')
    for  i in range(len(comb)):
        CRTS_coords[i] = ra_hms_split[i]+' '+dec_hms_split[i]
    CRTS_SkyCoords = SkyCoord(CRTS_coords, unit=(u.hourangle, u.deg))

    cat1 = CRTS_SkyCoords
else : 
    cat1_ra  = crts_qso['ra_CRTS']
    cat1_dec = crts_qso['dec_CRTS']
    cat1 = SkyCoord(ra=cat1_ra*u.degree, dec=cat1_dec*u.degree)
    
# Merge positionally the two catalogs with AStropy 
cat2_ra  = sdss_qso['ra']
cat2_dec = sdss_qso['dec']

cat2 = SkyCoord(ra=cat2_ra*u.degree, dec=cat2_dec*u.degree)
idx, sep2d, dist3d = cat1.match_to_catalog_sky(cat2) 

match_angle_deg = sep2d.value
match_angle_arcsec = match_angle_deg * 3600

# The fifteen outliers...
print('Number of those CRTS QSO without an SDSS match within 15 arcsec : %d' %np.sum(match_angle_arcsec> 15))

# take only those rows that are a perfect positional match to our catalog ... 
# even though we didn't have to do this, the fact that some stars share ra or dec made it hard 
# to use join() 
sdss_qso_table = sdss_qso[idx]

match_angle = Column(name='match_angle_arcsec',data= match_angle_arcsec)
sdss_qso_table.add_column(match_angle)

# add the sdss info to the crts info : stack two tables horizontally 
sdss_crts_table = hstack([crts_qso, sdss_qso_table])

# Display the fifteen outliers...
m = sdss_crts_table['match_angle_arcsec'] > 1
print('The 15 outliers.... ')
print(sdss_crts_table[m])

# Save the combined CRTS - SDSS table for quasars ... 
address = '../data_products/CRTS_SDSS_combined_QSO_catalog.dat'
sdss_crts_table.write(address, format='ascii', overwrite='True')

Number of those CRTS QSO without an SDSS match within 15 arcsec : 15
The 15 outliers.... 
raw_timespan_obs  raw_mean_err  raw_lc_length ...   Au  match_angle_arcsec
---------------- -------------- ------------- ... ----- ------------------
      2996.86109 0.209255319149         282.0 ... 0.137      351.674559391
      2996.86449        0.24195         200.0 ... 0.207      186.993884728
      2933.01351 0.145868852459         305.0 ... 0.153      271.525022433
      2917.05774 0.253652173913         115.0 ... 0.142      87.7045721106
      2938.08685 0.114838709677         279.0 ... 0.143      450.192877296
      2914.09165 0.343448275862          58.0 ... 0.371      247.422614573
      2849.23653 0.364444444444          45.0 ... 0.484      144.748025985
      2938.09008 0.337542372881         118.0 ... 0.585      551.280989912
      2920.01124 0.246804979253         241.0 ... 0.637      213.254624003
      1924.83354 0.182708333333          48.0 ... 0.468      142.664246028
      3055

In [17]:
np.ravel(sdss_crts_table.colnames)

array(['raw_timespan_obs', 'raw_mean_err', 'raw_lc_length', 'raw_mean_mag',
       'CRTS_name', 'proc_mean_N_day', 'proc_lc_length', 'proc_mean_mag',
       'proc_mjd_span', 'proc_mean_err', 'proc_median_dt', 'ra_CRTS',
       'dec_CRTS', 'dbID', 'ra', 'dec', 'SDR5ID', 'M_i', 'M_i_corr',
       'redshift', 'mass_BH', 'Lbol', 'u', 'g', 'r', 'i', 'z', 'Au',
       'match_angle_arcsec'], 
      dtype='<U18')

In [198]:
# Make an ascii file with list of ra, dec in degrees, separated by white space, commas (,), or pipes (|). 
# RA and Dec must be in units decimal degrees.

# first, feed the SDSS DR12 with the SDSS coords of mismatched QSO : 
radec_SDSS_mismatch = sdss_crts_table1[sdss_crts_table['match_angle_arcsec']>2][['ra','dec']]
address = '../data_products/CRTS_SDSS_mismatch_QSO_SDSS_radec.dat'
radec_SDSS_mismatch.write(address, format='ascii', overwrite='True')

# Feed the SDSS with the CRTS coords of mismatched objects .... 

radec_CRTS_mismatch = sdss_crts_table1[sdss_crts_table1['match_angle_arcsec']>2][['ra_CRTS','dec_CRTS']]
address = '../data_products/CRTS_SDSS_mismatch_QSO_CRTS_radec.dat'
radec_CRTS_mismatch.write(address, format='ascii', overwrite='True')

 Could do https://dr12.sdss.org/bulkFields/raDecs ,but it does not give much useful info...

 Instead, by hand try http://skyserver.sdss.org/dr7/en/tools/explore/obj.asp

 NOTES on individual objects : 

 First, I use  SDSS coords from DB9  to query by hand these 15 objects...

1) SDSS STAR ra=14.16039248, dec=0.61147335,   ObjId = 588015509812412634 : 
    
2) SDSS GALAXY ra=16.57797276, dec=-0.36779433,   ObjId = 588015508739719362

3) SDSS STAR    ra=21.79044991, dec=-0.82312779,   ObjId = 588015508205142178

4) SDSS STAR    ra=23.57580185, dec=0.26019066,   ObjId = 587731513144770582

5) SDSS STAR    ra=30.27577764, dec=-0.9485065,   ObjId = 587731511537041576

6) SDSS STAR    ra=43.79397299, dec=-0.12030369,   ObjId = 587731512616747105

7) SDSS STAR    ra=44.02973877, dec=1.07722393,   ObjId = 587731514227425430

8) SDSS STAR    ra=51.85132224, dec=-0.85782689,   ObjId = 587731511546478754

9) SDSS STAR    ra=55.73044693, dec=0.8231733,   ObjId = 587731513695666438

10) SDSS STAR    ra=311.17075564, dec=0.94198223,   ObjId = 587730848497206143
--> crowded field 

11) SDSS STAR    ra=335.40398149, dec=0.19558045,   ObjId = 587734304878690655

12) SDSS STAR    ra=343.92565411, dec=-1.12342469,   ObjId = 587734303271813146

13) SDSS STAR    ra=348.04136769, dec=-0.94963622,   ObjId = 587731185124966472

14) SDSS STAR    ra=350.34892733, dec=-0.02322992,   ObjId = 587731186199691464

15) SDSS STAR    ra=355.04130819, dec=0.93886394,   ObjId = 588015510340960369

--> some looked like QSO, but then got spectra, but not all have spectra! 
--> this is surprising... well, I was looking at DR7, is there anything more possibly in DR9 ? 

--> DR12 catalog is not easy to browse... 

--> Here in DR13 I can paste ra, dec and get poststamps that can be further inspected ... 


## Exec. summary on the CRTS-SDSS mismatched QSO ...

These 15 objects according to DB9 are QSO, and indeed they are. The CRTS objects that are mismatched sometimes are quasars, sometimes they are not.... I wonder where Brani took the QSO from. He said they were spectroscopically confirmed. All mismatches are 'catastrophic', i.e. more than 50 arcsec separation. 

Possible problems: 
   - are we using different coordinate systems between CRTS and SDSS ? The default frame in SkyCoord is ICRS   ("The International Celestial Reference System (ICRS) is the current standard celestial reference system adopted by the International Astronomical Union (IAU). Its origin is at the barycenter of the Solar System, with axes that are intended to be "fixed" with respect to space.")
   - problems with hms --> deg conversion : unlikely. I checked the matching in two ways, and results still persist, with exactly the same 15 quasars being mismatched (without an SDSS counterpart within 2 arcsec).  